In [47]:
import os
import pandas as pd
import numpy as np
import altair as alt

from scipy.stats import fisher_exact
from math import log


In [17]:
TSS_bedcols = ['chr', 'TSS500up', 'TSS500down', 'Gene_Name']

In [3]:
def __simi_mat(tflist, escape_dir):
    
    mat = pd.DataFrame(index=tflist, columns=tflist)
    dice_mat = pd.DataFrame(index=tflist, columns=tflist)
    
    tflist_pop = tflist.copy()
    for t in tflist:
        tfile = pd.read_table(os.path.join(escape_dir, t + ".bed.gz"), names=TSS_bedcols)
        #print('t =', t)s

        if tfile.shape[0] == 0:
            mat.loc[t] = np.nan
            mat.loc[:, t] = np.nan

            dice_mat.loc[t] = np.nan
            dice_mat.loc[:, t] = np.nan

            tflist_pop.pop(0)
            continue

        for t2 in tflist_pop:
            t2file = pd.read_table(os.path.join(escape_dir, t2 + ".bed.gz"), names=TSS_bedcols)
            #print('t2 = ', t2)

            if t2file.shape[0] == 0:
                mat.loc[t, t2] = np.nan
                mat.loc[t2, t] = np.nan

                dice_mat.loc[t, t2] = np.nan
                dice_mat.loc[t2, t] = np.nan

            else:
                inner = tfile.merge(t2file, how='inner').shape[0]

                union_size = tfile.shape[0] + t2file.shape[0] - inner
                sim = inner / union_size
                mat.loc[t, t2] = sim
                mat.loc[t2, t] = sim

                dice_sim = (2 * inner) / (tfile.shape[0] + t2file.shape[0])
                dice_mat.loc[t, t2] = dice_sim
                dice_mat.loc[t2, t] = dice_sim


        tflist_pop.pop(0)
    
    mat = mat.astype(float)
    np.fill_diagonal(mat.values, 1.01)

    dice_mat = dice_mat.astype(float)
    np.fill_diagonal(dice_mat.values, 1.01)
        
    return mat, dice_mat

In [38]:
meta = pd.read_table('./db/meta_tfs.tsv')
remap2022_TFs = meta['TF_Name'].tolist()

escapeoverlapdir = './ReMap2022_Overlap/ReMap2022_Escape/'
bgoverlapdir = './ReMap2022_Overlap/ReMap2022_BG/'

tss_summary = pd.read_table('./summary/tss_overlapescape_2022.tsv', sep='\t')
remap2022_TSStfs = tss_summary.query('enriched == True')['TF'].tolist()
remap2022_strongTFs = ['HSF1', 'ZFP36', 'NIPBL', 'MYB', 'STAT1']

In [6]:
tf_pairs = []
tflist2 = remap2022_TFs.copy()
for tf in remap2022_TFs:
    tflist2.pop(0)
    for tf1 in tflist2:
        s = sorted([tf, tf1])
        tf_pairs.append(s)

In [30]:
sim_allvsall, dice_allvsall = __simi_mat(remap2022_TFs, escapeoverlapdir)
hm_simava = sim_allvsall.melt(ignore_index=False).reset_index()
hm_diceava = dice_allvsall.melt(ignore_index=False).reset_index()

In [31]:
alt.hconcat(
    alt.Chart(hm_simava.query('variable == @remap2022_TSStfs & index == @remap2022_TSStfs')).mark_rect().encode(
    alt.X('variable', title='TFs'),
    alt.Y('index', title="TFs"),
    alt.Color('value', title='index'),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=300, width=300),
    alt.Chart(hm_diceava.query('variable == @remap2022_TSStfs & index == @remap2022_TSStfs')).mark_rect().encode(
    alt.X('variable', title='TFs'),
    alt.Y('index', title="TFs"),
    alt.Color('value', title='index'),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=300, width=300)
)

alt.HConcatChart(...)

In [33]:
sim_bg_allvsall, dice_bg_allvsall = __simi_mat(remap2022_TFs, bgoverlapdir)
hm_simbgava = sim_bg_allvsall.melt(ignore_index=False).reset_index()
hm_dicebgava = dice_bg_allvsall.melt(ignore_index=False).reset_index()

In [34]:
alt.hconcat(
alt.Chart(hm_simbgava.query('variable == @remap2022_TSStfs & index == @remap2022_TSStfs')).mark_rect().encode(
    alt.X('variable', title='TFs'),
    alt.Y('index', title="TFs"),
    alt.Color('value', title='index'),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=300, width=300), 
alt.Chart(hm_dicebgava.query('variable == @remap2022_TSStfs & index == @remap2022_TSStfs')).mark_rect().encode(
    alt.X('variable', title='TFs'),
    alt.Y('index', title="TFs"),
    alt.Color('value', title='index'),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=300, width=300)
)

alt.HConcatChart(...)

In [40]:
sim_foreoverbg = sim_allvsall / sim_bg_allvsall
dice_foreoverbg = dice_allvsall / dice_bg_allvsall
hm_esoverbg = sim_foreoverbg.melt(ignore_index=False).reset_index()
hm_diceesoverbg = dice_foreoverbg.melt(ignore_index=False).reset_index()

alt.hconcat(
    alt.Chart(hm_esoverbg.query('variable == @remap2022_TSStfs & index == @remap2022_strongTFs')).mark_rect().encode(
    alt.X('variable', title='TFs'),
    alt.Y('index', title="TFs"),
    alt.Color('value', title='es/bg'),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=300, width=300),
alt.Chart(hm_diceesoverbg.query('variable == @remap2022_TSStfs & index == @remap2022_strongTFs')).mark_rect().encode(
    alt.X('variable', title='TFs'),
    alt.Y('index', title="TFs"),
    alt.Color('value', title='es/bg'),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=300, width=300)
)

alt.HConcatChart(...)

In [41]:
alt.Chart(hm_diceesoverbg.query('variable == @remap2022_TSStfs & index == @remap2022_TSStfs')).mark_rect().encode(
    alt.X('variable', title='', axis=alt.Axis(labelFontSize=25, titleFontSize=25)),
    alt.Y('index', title="" , axis=alt.Axis(labelFontSize=25, titleFontSize=25)),
    alt.Color('value', title='escape/background', scale=alt.Scale(scheme='lightgreyred')),
    alt.Tooltip(['index', 'variable','value'])
).properties(height=800, width=800).configure_title(fontSize=40, anchor='start').configure_axis(
    labelFontSize=25, 
    titleFontSize=25,
).configure_legend(
titleFontSize=20,
labelFontSize=20,
gradientLength=250,
gradientThickness=40,
titleLimit=200) 

alt.Chart(...)

### SUMMARY - Stats

In [43]:
def __stat_mat(tflist, escape_dir, total_size):
    fisher_mat = pd.DataFrame(index=tflist, columns=tflist)

    tflist_pop = tflist.copy()
    for t in tflist:
        tfile = pd.read_table(os.path.join(escape_dir, t + ".bed.gz"), names=TSS_bedcols)
        tfile_size = tfile.shape[0]

        if tfile_size == 0:
            fisher_mat.loc[t] = np.nan
            fisher_mat.loc[:, t] = np.nan

            tflist_pop.pop(0)
            continue

        for t2 in tflist_pop:
            chi_df = pd.DataFrame(index=['A_B', 'A_notB', 'notA_B', "notA_notB"], columns=['obs', 'exp'])
            t2file = pd.read_table(os.path.join(escape_dir, t2 + ".bed.gz"), names=TSS_bedcols)
            t2file_size = t2file.shape[0]
            #print('t2 = ', t2)

            if t2file_size == 0:
                fisher_mat.loc[t, t2] = np.nan
                fisher_mat.loc[t2, t] = np.nan

            else:
                inner = tfile.merge(t2file, how='inner').shape[0]

                A_only = tfile_size - inner
                B_only = t2file_size - inner

                chi_df.loc['A_B', 'obs'] = inner
                chi_df.loc['A_notB', 'obs'] = A_only
                chi_df.loc['B_notA', 'obs'] = B_only
                chi_df.loc['notA_notB', 'obs'] = total_size - inner - A_only - B_only
                chi_df.loc['A_B', 'exp'] = tfile_size * t2file_size / total_size
                chi_df.loc['A_notB', 'exp'] = tfile_size * (total_size - t2file_size) / total_size
                chi_df.loc['B_notA', 'exp'] = t2file_size * (total_size - tfile_size) / total_size
                chi_df.loc['notA_notB', 'exp'] = (total_size - t2file_size) * (total_size - tfile_size) / total_size

                odds, fisher_pval = fisher_exact([[inner, A_only], [B_only, chi_df.loc['notA_notB', 'obs']]])                
                fisher_mat.loc[t, t2] = fisher_pval
                fisher_mat.loc[t2, t] = fisher_pval

        tflist_pop.pop(0)

    fisher_mat = fisher_mat.astype(float)
    np.fill_diagonal(fisher_mat.values, 1.01)

    return fisher_mat

In [44]:
fe_allvsall = __stat_mat(remap2022_TFs, escapeoverlapdir, 55)
fe_bg_allvsall = __stat_mat(remap2022_TFs, bgoverlapdir, 275)

In [46]:
tf_pairs_sum = pd.DataFrame({'pairs':tf_pairs}).sort_values('pairs')

FDR = 0.10

for i, row in tf_pairs_sum.iterrows():
    p = row[0]
    tf_pairs_sum.loc[i, 'tf1'] = p[0]
    tf_pairs_sum.loc[i, 'tf2'] = p[1]
    tf_pairs_sum.loc[i, 'sim_esoverbg'] = sim_foreoverbg.loc[p[0], p[1]]
    tf_pairs_sum.loc[i, 'dice_esoverbg'] = dice_foreoverbg.loc[p[0], p[1]]

    tf_pairs_sum.loc[i, 'es_fepval'] = fe_allvsall.loc[p[0], p[1]]
    tf_pairs_sum.loc[i, 'bg_fepval'] = fe_bg_allvsall.loc[p[0], p[1]]

tf_pairs_sum = tf_pairs_sum.merge(meta[['TF_Name', 'protein_class']], left_on='tf1', right_on='TF_Name', how='left').drop(columns=['TF_Name']).rename(columns={'protein_class':'tf1_class'})
tf_pairs_sum = tf_pairs_sum.merge(meta[['TF_Name', 'protein_class']], left_on='tf2', right_on='TF_Name', how='left').drop(columns=['TF_Name']).rename(columns={'protein_class':'tf2_class'})
tf_pairs_sum

tf_pairs_sum['esfepval_rank'] = tf_pairs_sum[['es_fepval']].rank(method="min")
tf_pairs_sum['bgfepval_rank'] = tf_pairs_sum[['bg_fepval']].rank(method="min")
tf_pairs_sum['BH_esfepval'] = tf_pairs_sum['esfepval_rank'] / tf_pairs_sum['esfepval_rank'].max() * FDR

tf_pairs_sum['BH_bgfepval'] = tf_pairs_sum['bgfepval_rank'].max() / tf_pairs_sum['bgfepval_rank'] * tf_pairs_sum['bg_fepval']

clean_cols = ['pairs', 'tf1', 'tf2', 'tf1_class', 'tf2_class', 'sim_esoverbg', 'dice_esoverbg', 'es_fepval', 'BH_esfepval', 'BH_bgfepval']

In [48]:
tf_pairs_sum['log_BH_esfepval'] = pd.Series(map(np.log10, tf_pairs_sum['BH_esfepval'])) * -1
tf_pairs_sum['log_BH_bgfepval'] = pd.Series(map(np.log10, tf_pairs_sum['BH_bgfepval'])) * -1

In [54]:
strongpairs = tf_pairs_sum.query('tf1 == @remap2022_TSStfs & tf2 == @remap2022_TSStfs')[clean_cols].query('tf1 == @remap2022_strongTFs | tf2 == @remap2022_strongTFs').query('dice_esoverbg >= 1.5 & BH_esfepval > es_fepval')
strongpairs['tf1'].append(strongpairs['tf2']).value_counts()
strongpairs.sort_values('dice_esoverbg', ascending=False)

,pairs,tf1,tf2,tf1_class,tf2_class,sim_esoverbg,dice_esoverbg,es_fepval,BH_esfepval,BH_bgfepval
6970,"[IRF5, ZFP36]",IRF5,ZFP36,winged helix/forkhead transcription factor,RNA metabolism protein,2.705882,2.380952,2.165944e-02,0.045231,3.403370e-02
6460,"[HSF1, ZFP36]",HSF1,ZFP36,winged helix/forkhead transcription factor,RNA metabolism protein,2.750000,2.166667,7.349317e-05,0.016504,1.473626e-03
2190,"[BRCA1, ZFP36]",BRCA1,ZFP36,ubiquitin-protein ligase,RNA metabolism protein,2.500000,2.142857,1.255619e-03,0.026219,8.049930e-03
8415,"[MYB, TBP]",MYB,TBP,NaN,general transcription factor,2.650000,2.100000,1.192139e-04,0.017707,7.960019e-04
7360,"[MAFF, ZFP36]",MAFF,ZFP36,basic leucine zipper transcription factor,RNA metabolism protein,2.294118,1.956522,1.595405e-03,0.027473,4.411313e-03
2130,"[BRCA1, NIPBL]",BRCA1,NIPBL,ubiquitin-protein ligase,"chromatin/chromatin-binding, or -regulatory pr...",2.285714,1.947368,5.754923e-04,0.023141,4.456554e-03
11100,"[STAT1, ZFP36]",STAT1,ZFP36,DNA-binding transcription factor,RNA metabolism protein,2.296651,1.849530,5.271102e-05,0.015605,7.109672e-06
9173,"[NIPBL, RBBP5]",NIPBL,RBBP5,"chromatin/chromatin-binding, or -regulatory pr...",NaN,2.071429,1.833333,2.935010e-03,0.030589,5.958426e-03
6455,"[HSF1, ZBED1]",HSF1,ZBED1,winged helix/forkhead transcription factor,DNA metabolism protein,2.214286,1.809524,1.017998e-04,0.017213,1.365080e-04
11620,"[UBTF, ZFP36]",UBTF,ZFP36,DNA-binding transcription factor,RNA metabolism protein,2.087719,1.794872,3.916112e-03,0.032046,8.454345e-04
